## 4週目、4日目へようこそ

これが素晴らしいプロジェクトの始まりです！本当にシンプルで非常に効果的です。

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
load_dotenv(override=True)

### 非同期のランググラフ

ツールを実行するには：  
同期： `tool.run（inputs）`  
async： `await tool.arun（inputs）`

グラフを呼び出すには：  
同期： `graph.invoke（state）`  
async： `await graph.ainvoke（state）`

In [3]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

## 追加のインストールステップ - コンピューターにノードと劇作家がない場合

次に、まだ持っていない場合は、コンピューターにnodejsとplaywrightをインストールする必要があります。ここで手順をご覧ください：

[Node and Playwright setup](../setup/SETUP-node.md)

## そして今 -  Playwrightをインストールした後、Windows PCユーザー用のヘッドアップ：

次のいくつかのセルを実行している間、PlaywrightブラウザがnotimplementedErrorを上げていることに問題が発生する可能性があります。

これは、Pythonモジュールに移動するときに機能するはずですが、ノートブックのWindowsに問題を引き起こす可能性があります。

このエラーがあり、ノートブックを実行したい場合は、非常にハッキーな小さな変更を行う必要があります。 Playwright（以前のセル）を設置した後、これを行う必要があります

1.左側のファイルエクスプローラーで `.venv`を右クリックし、「フォルダーで検索」を選択します
2。 `asyncio.set_event_loop_policy（windowsselectoreventlooppolicy（））`を検索します  
3。そのコードは、 `kernelapp.py`というファイルのコードの行で見つける必要があります
4.この行がの一部であることを他の条項全体にコメントしてください - 以下のフラグメントを参照してください。輸入業者ラインの後に「パス」ステートメントを持っているようにしてください。
5。上の「再起動」ボタンを押してカーネルを再起動します

```python
        if sys.platform.startswith("win") and sys.version_info >= (3, 8):
            import asyncio
 
            try:
                from asyncio import WindowsProactorEventLoopPolicy, WindowsSelectorEventLoopPolicy
            except ImportError:
                pass
                # not affected
           # else:
            #    if type(asyncio.get_event_loop_policy()) is WindowsProactorEventLoopPolicy:
                    # WindowsProactorEventLoopPolicy is not compatible with tornado 6
                    # fallback to the pre-3.8 default of Selector
                    # asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
```

これを見つけてくれた学生ニコラス、そしてカリヤン、ヤキ、ジビン、バスカーに感謝します。そして、余分なポインターのためにvladislavに。

別の方法として、Pythonモジュールに移動することができます（とにかく5日目に実行します）

In [5]:
# nest_asyncioの紹介
# Python Asyncコードでは、1つの「イベントループ」を処理するイベントのみを処理できます。
# `nest_asyncio`ライブラリはこれをパッチし、ネストされたイベントループを実行する必要がある場合は、特別な状況に使用されます。

import nest_asyncio
nest_asyncio.apply()

### Langchainコミュニティ

Langchainの驚くべきことの1つは、その周りの豊かなコミュニティです。

これをチェックしてください：


In [6]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# ここまたは後でnotimplementederrorを入手した場合は、ノートブックの上部にあるヘッズをご覧ください

async_browser =  create_async_playwright_browser(headless=False)  # ヘッドフルモード
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [ ]:
for tool in tools:
    print(f"{tool.name}={tool}")

In [8]:
tool_dict = {tool.name:tool for tool in tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

    
await navigate_tool.arun({"url": "https://www.cnn.com"})
text = await extract_text_tool.arun({})

In [ ]:
import textwrap
print(textwrap.fill(text))

In [17]:
all_tools = tools + [tool_push]

In [18]:

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [ ]:

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "10"}}

async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()